# Text Classification
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. Please check the pdf file for more details.*

In this exercise you will:
    
- implement a of spam classifier with **Naive Bayes method** for real world email messages
- learn the **training and testing phase** for Naive Bayes classifier  
- get an idea of the **precision-recall** tradeoff

In [1]:
# some basic imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
# ham_train contains the occurrences of each word in ham emails. 1-by-N vector
ham_train = np.loadtxt('ham_train.csv', delimiter=',')
# spam_train contains the occurrences of each word in spam emails. 1-by-N vector
spam_train = np.loadtxt('spam_train.csv', delimiter=',')
# N is the size of vocabulary.
N = ham_train.shape[0]
# There 9034 ham emails and 3372 spam emails in the training samples
num_ham_train = 9034
num_spam_train = 3372
# Do smoothing
x = np.vstack([ham_train, spam_train]) + 1

# ham_test contains the occurences of each word in each ham test email. P-by-N vector, with P is number of ham test emails.
i,j,ham_test = np.loadtxt('ham_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
ham_test_tight = scipy.sparse.coo_matrix((ham_test, (i - 1, j - 1)))
ham_test = scipy.sparse.csr_matrix((ham_test_tight.shape[0], ham_train.shape[0]))
ham_test[:, 0:ham_test_tight.shape[1]] = ham_test_tight
# spam_test contains the occurences of each word in each spam test email. Q-by-N vector, with Q is number of spam test emails.
i,j,spam_test = np.loadtxt('spam_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
spam_test_tight = scipy.sparse.csr_matrix((spam_test, (i - 1, j - 1)))
spam_test = scipy.sparse.csr_matrix((spam_test_tight.shape[0], spam_train.shape[0]))
spam_test[:, 0:spam_test_tight.shape[1]] = spam_test_tight


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


## Now let's implement a ham/spam email classifier. Please refer to the PDF file for details

In [3]:
from likelihood import likelihood
# TODO
# Implement a ham/spam email classifier, and calculate the accuracy of your classifier

# Hint: you can directly do matrix multiply between scipy.sparse.coo_matrix and numpy.array.
# Specifically, you can use sparse_matrix * np_array to do this. Note that when you use "*" operator
# between numpy array, this is typically an elementwise multiply.

# begin answer

# likelihood

l = likelihood(x)

##### top 10 SPAM words #####

# find index

ratio = l[0, :] / l[1, :] # reverse ratio
ratio_sorted = np.sort(ratio) # find the 10 smallest
ind = []
for i in range(10):
    ind.append([j for j, r in enumerate(ratio) if r == ratio_sorted[i]]) # there might be ties

# look up words by index

file = open('all_word_map.txt', 'r')
lines = []
for line in file.readlines():
    lines.append(line.split())
print('top 10 SPAM words:')
for i in range(10):
    for j in ind[i]:
        print('%2d' % (i+1), '%6s' % lines[j][1], '%10s' % lines[j][0], '%6.f' % (1/ratio_sorted[i]))

##### classify & test #####

# log-likelihood: to avoid floating underflow

# test set 1: labeled HAM
# test set 2: labeled SPAM

l_log = np.log(l)
l_set1 = ham_test * l_log.T # P(wi=N|SPAM)=P(wi|SPAM)^N
l_set2 = spam_test * l_log.T
l_ham_set1 = l_set1[:, 0]
l_ham_set2 = l_set2[:, 0]
l_spam_set1 = l_set1[:, 1]
l_spam_set2 = l_set2[:, 1]

# prior

num_total_train = num_ham_train + num_spam_train
prior_ham = np.log(num_ham_train / num_total_train)
prior_spam = np.log(num_spam_train / num_total_train)

# posterior

post_ham_set1 = l_ham_set1 + prior_ham
post_ham_set2 = l_ham_set2 + prior_ham
post_spam_set1 = l_spam_set1 + prior_spam
post_spam_set2 = l_spam_set2 + prior_spam

# confusion matrix

TP = np.sum([1 for i in range(spam_test.shape[0]) if post_spam_set2[i] > post_ham_set2[i]]) # true for SPAM
TN = np.sum([1 for i in range(ham_test.shape[0]) if post_ham_set1[i] > post_spam_set1[i]]) # true for HAM
FN = spam_test.shape[0] - TP
FP = ham_test.shape[0] - TN
print('TP = ', TP, '  FP = ', FP)
print('FN = ', FN, '  TN = ', TN)

# accuracy & precision & recall

print('accuracy: %.4f' % ((TP + TN) / (spam_test.shape[0] + ham_test.shape[0])))
print('precision = %.4f' % (TP / (TP + FP)))
print('recall = %.4f' % (TP / (TP + FN)))

# end answer


top 10 SPAM words:
 1  30033       nbsp   1325
 2  75526     viagra   1250
 3  38176      pills   1102
 4  45153     cialis    848
 5   9494       voip    838
 6  65398        php    769
 7  37568       meds    673
 8  13613  computron    652
 9  56930        sex    614
10   9453     ooking    518
10  19957      width    518
TP =  1093   FP =  28
FN =  31   TN =  2983
accuracy: 0.9857
precision = 0.9750
recall = 0.9724
